# ENV

In [211]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:78% !important; }</style>"))

In [209]:
import pandas as pd
import numpy as np
from datetime import datetime, date


import json

import matplotlib as mpl
import matplotlib.pylab as plt

In [210]:
import xlsxwriter

# read_from_Excel

In [251]:
df = pd.read_excel("data/stock_price_example.xlsx", sheet_name = "stock_price_example")

In [12]:
df2 = df[['date','close']]

In [13]:
df2

,date,close
0,2019-01-02,38750
1,2019-01-03,37600
2,2019-01-04,37450
3,2019-01-07,38750
4,2019-01-08,38100
...,...,...
107,2019-06-11,44850
108,2019-06-12,44600
109,2019-06-13,43750
110,2019-06-14,44000


# write_to_Excel

In [14]:
writer = pd.ExcelWriter('data/df2.xlsx', engine = 'xlsxwriter')

In [15]:
df2.to_excel(writer, sheet_name = 'test')

In [16]:
writer.save()

* 아래와 같이 함수로 만들어 두면 편리

In [36]:
def excel_save(df,file_name,sheet_name):
    writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')
    df.to_excel(writer, sheet_name = sheet_name)
    writer.save()
    


In [ ]:
excel_save(df2, 'data/df3.xlsx','test2')

# Excel_Chart

* 참조: https://xlsxwriter.readthedocs.io/chart.html

* excel 차트 생성을 위해서는 excel 고유의 workbook, worksheet이 정의되어야 함

* 먼저 차트 작성을 위한 데이터를 생성합니다.

In [105]:
df_chart = df.copy()
df_chart = df_chart[['date','close']]
df_chart_mean = df_chart.groupby(df_chart['date'].dt.month).mean()

In [106]:
df_chart_mean

,close
date,
1,41600.000000
2,46308.823529
3,44560.000000
4,46061.363636
5,43023.809524
6,44125.000000


* 이제 이 데이터를 기반으로 하는 excel 개체를 생성합니다.

In [107]:
writer = pd.ExcelWriter('data/mychart.xlsx', engine = 'xlsxwriter')
df_chart_mean.to_excel(writer, sheet_name = 'mysheet')
workbook = writer.book
worksheet = writer.sheets['mysheet']

* workbook의 add_chart 함수로 chart 개체를 생성합니다.

In [108]:
chart = workbook.add_chart({'type': 'column'})

* 이제 이렇게 생성된 차트 개체에 데이터 및 서식을 지정해 줍니다.

In [109]:
# 데이터 지정
chart.add_series({'values' : '=mysheet!$B$2:$B$7'})
# 차트의 범례 위치 지정
chart.set_legend({'position':'none'})
# 차트 삽입 위치 지정
worksheet.insert_chart('D2', chart)


* 완성된 excel 파일을 저장

In [104]:
writer.save()

# Cell_Style

* cell 서식을 지정
* 참조: https://xlsxwriter.readthedocs.io/format.html

In [113]:
df2

,date,close
0,2019-01-02,38750
1,2019-01-03,37600
2,2019-01-04,37450
3,2019-01-07,38750
4,2019-01-08,38100
...,...,...
107,2019-06-11,44850
108,2019-06-12,44600
109,2019-06-13,43750
110,2019-06-14,44000


In [141]:
writer = pd.ExcelWriter("data/df2_cell_style.xlsx", engine='xlsxwriter')

# 데이터프레임의 객체를 XlsxWriter Excel 객체로 변환 
df2.to_excel(writer, sheet_name='test')

# xlsxwriter workbook and worksheet 객체 호출
workbook  = writer.book
worksheet = writer.sheets['test']

In [142]:

# Cell의 포맷팅 설정 
# format_date = workbook.add_format({'num_format': 'yyyy-mm'})
format_amt = workbook.add_format({'num_format': '#,##0'})

# B:B 전체, Cell 크기 20, format 설정은 위의 format1
# worksheet.set_column('B:B', 20, format_date)

# C:C 전체, Cell 크기 기본값 , format 설정은 위의 format2
worksheet.set_column('C:C', 15, format_amt)
format_amt.set_font_color('blue')


In [143]:
writer.save()

# 자동처리 예시

In [231]:
df5 = df.copy()

In [232]:
df5 = df5[['date','close','volume']]

In [217]:
df5['month'] = df5['date'].dt.month

In [250]:
# 객체 생성
writer = pd.ExcelWriter('data/multi_sheet.xlsx', engine = 'xlsxwriter')

for x in df5['date'].dt.month.unique():# 월별 데이터 및 차트 생성
    
    # df 생성 : 월단위로 추출
    df6 = df5[df5['date'].dt.month == x]
    
    # sheet name 정의
    s_name = str(x) + '월'
    
    # object 생성    
    df6.to_excel(writer, sheet_name = s_name)
    workbook = writer.book
    worksheet = writer.sheets[s_name]
    
#     # line 차트 생성
    line_chart = workbook.add_chart({'type': 'line'})
    
    # 데이터가 존재하는 범위를 설정하기 위한 변수 생성
    end_cell_position = df6.shape[0] + 1
    value_scope = "'" + s_name + "!$C$2:$C$" + str(end_cell_position) + "'"
    
    # 데이터가 존재하는 범위를 설정하여 seriea 구성
    line_chart.add_series({'values' : value_scope})
    
    # 차트 삽입
    worksheet.insert_chart('E2', line_chart)
    
# 복수개의 시트가 추가된 최종 파일 완성 후 loop를 벗어나서 저장
writer.save()


In [200]:
writer = pd.ExcelWriter('data/stock_chart.xlsx', engine = 'xlsxwriter')
df5.to_excel(writer, sheet_name = 'sheet')
workbook = writer.book
worksheet = writer.sheets['sheet']

In [201]:
column_chart = workbook.add_chart({'type': 'column'})
column_chart.add_series({'values' : '=sheet!$C$2:$C$111'})

In [202]:
line_chart = workbook.add_chart({'type': 'line'})
line_chart.add_series({'values' : '=sheet!$D$2:$D$111','y2_axis': True})

In [203]:
column_chart.combine(line_chart)

In [204]:
worksheet.insert_chart('E2', column_chart)

In [205]:
writer.save()